# Encoding Data Class

Present notebook reviews the **Encoding** class from the *encoding_protocols* module from the **QQuantLib.DL.encoding_protocols** file.

The **Encoding** class is a python one which main objective is create a quantum circuit for encoding some input data, in a  transparent way for the user of the library. The output quantum circuit can be used as an oracle for **Amplitude Estimation** problems. In this class up to 3 different encoding methods were implemented.

In [ ]:
import sys
sys.path.append("../../")
import numpy as np
import pandas as pd
import qat.lang.AQASM as qlm
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import integrate

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
from QQuantLib.utils.data_extracting  import get_results

## 1. Encoding Problem

Main objective behind the **Encoding** class is create, in a transparent way for the user, a quantum oracle circuit that can be used for computing integrals using **Amplitude Estimation** algorithms. 

Let $p(x)$ a probability distribution and $f(x)$ a function defined over an interval $[a, b] \subset \mathbf{R}$, our main idea is compute the expected value of the function $f(x)$ when $x$ follows a probability distribution $p(x)$ between the interval $[a, b]$ using **AE** techniques (and of course the **AE** class created in the **QQuantLib.AE.ae_class**). The expected value can be computed as the following integral:

$$I = \int_a^bp(x)f(x)dx$$

 
In this case we are going to approximate this integral as a Riemman sum so:


$$P=\{[x_0, x_1], [x_1, x_2], ..., [x_{n-1}, x_n]\}$$ 

such that:

$$a = x_0 < x_1 < x_2 < ... < x_n=b$$

Then 

$$I= \int_a^bp(x)f(x)dx \approx S = \sum_i^np(x_i)f(x_i)*\Delta x_i$$

The final objective is create a quantum circuit, where the before Riemman sum is codified into the amplitude of one of the *eigen-state* of the $|\Psi\rangle$ state.

For doing this several algorithms were identified and implemented in our Python class.

The $p(x)$ and the $f(x)$ functions will be encoding as an arrays so a discretization is needed. 


### Instantiate the class

For creating the encoding quantum circuit we can instantiate the **Encoding** class from the *encoding_protocols* module. This class have following inputs:

* *array_function*: numpy array with the discretization of the of the function $f(x)$. This is mandatory.
* *array_probability*: numpy array with the discretization of the probability $p(x)$. This can be None, then the uniform distribution will be used.
* *encoding*: integer for indicating the encoding procedure to use.

Additionally, a python dictionary with other configuration settings can be provided to the class. Main  key of this dictionary will be:

* *multiplexor*: boolean variable for using (or not) multiplexors.

Additonally some attributes are created when class is instantiated:

* *oracle*: where the *QLM AbstractGate* of the desired oracle will be stored.
* *co_target*:  list of ints with the state the oracle marks in binnary representation.
* *co_index*: list of ints with the registers over the oracle will act.
* *p_gate*; where the *QLM AbstractGate* of the operator $\mathbf{U}_p$ will be stored.
* *function_gate*; where the *QLM AbstractGate* of the operator $\mathbf{U}_f$ will be stored.
* **encoding_normalization**: this will be the normalisation due to the encoding procedure. So for recover the integral the amplitude of the state where we want to codified it we need **ALWWAYS** multuply the amplitude by this attribute!!

In [ ]:
from QQuantLib.DL.encoding_protocols import Encoding

### BE AWARE!! Input restrictions

There are some **MANDATORY** conditions that the numpy inputs arrays provided to the class shoud be satisfied:

* All the arrays must be of dimension $2^n$ where $n$ is an integer number. In this case the $n$ will be related with the number of qubits used for creating the quantum circuit.
* $f(x)$ **MUST BE** properly normalised: $f(x_i) \leq 1 \forall i$
* $p(x)$, if provided, **MUST BE** properly normalised: $\sum_{i=0}^{2^{n}} p(x_i) = 1$

If any of these conditions is not satisfied an Error will be raised!!!

In [ ]:
#Bad interval number
a = 0
b = 5.0
domain_y = np.linspace(a, b, 10)
#Our discretized function
f_y = domain_y*domain_y

#Our discretized probability distribution
p_y = domain_y

#Raise an error because bad interval number
class_encoding0 = Encoding(array_function=f_y, array_probability=p_y, encoding=0)

In [ ]:
#Bad function normalisation
a = 0
b = 5.0
n = 6

#domain 
domain_y = np.linspace(a, b, 2**n)

#Our discretized function
f_y = domain_y*domain_y

#Our discretized probability distribution
p_y = domain_y

#Raise an error because function is not properly normalised
class_encoding0 = Encoding(array_function=f_y, array_probability=p_y, encoding=0)

In [ ]:
#Bad probability normalisation
a = 0
b = 5.0
n = 6

#domain 
domain_y = np.linspace(a, b, 2**n)

#Our discretized function
f_y = domain_y*domain_y
f_y = f_y / np.max(f_y)
print(np.max(f_y))

#Our discretized probability distribution (bad one)
p_y = domain_y

#Raise an error because probability is not properly normalised
class_encoding0 = Encoding(array_function=f_y, array_probability=p_y, encoding=0)

In [ ]:
# All properly configured
a = 0
b = 5.0
n = 6

#domain 
domain_y = np.linspace(a, b, 2**n)

#Our discretized function
f_y = domain_y*domain_y
#normalisation for function
f_y = f_y / np.max(f_y)
print(np.max(f_y))

#Our discretized probability distribution
p_y = domain_y
#normalisation for probability
p_y = p_y / np.sum(p_y)

#Raise an error because probability is not properly normalised
class_encoding0 = Encoding(array_function=f_y, array_probability=p_y, encoding=0)

In last cell all was properly configured so not error are raised!! Now we can continue explaining the class. 

## 2. Encoding Procedure

3 different procedure were implemented in the class. Each procedure can be selected by giving a number between 0 and 2 to the encoding attribute of the class. When the encoding changes all the class atributes are reseted!!

First we created the mandatory data for encoding

In [ ]:
a = np.pi/4.0
b = np.pi/2.0
#number of qbits
n = 6
#Our domain
domain_x = np.linspace(a, b, 2**n)
#Our discretized probability distribution
p_x = domain_x
#Our discretized function
f_x = np.sin(domain_x)

In [ ]:
plt.plot(domain_x, p_x, 'o')
plt.plot(domain_x, f_x, 'o')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['p(x)', 'f(x)'])

In [ ]:
#Normalisations!!!
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation
print("Probability properly normalised?: {}".format(np.sum(norm_p_x) <= 1.0))
f_x_normalisation = np.max(f_x) +  1e-8
norm_f_x = f_x / f_x_normalisation
print("Function properly normalised?: {}".format(np.max(norm_f_x) <= 1.0))

In [ ]:
plt.plot(domain_x, norm_p_x, 'o')
plt.plot(domain_x, norm_f_x, 'o')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['p(x)', 'f(x)'])

In [ ]:
#Our idea is computing the following result:
Riemman = np.sum(norm_p_x*norm_f_x)

print("We want to load: {}".format(Riemman))

In [ ]:
#Testing the normalisations
np.sum(norm_p_x*norm_f_x) *  p_x_normalisation * f_x_normalisation == np.sum(p_x*f_x)

### 2.1 Encoding Procedure 0.

This encoding algorithm is the originally proposed by Grover and Rudolph:

* *Grover, Lov and Rudolph, Terry*. Creating superpositions that correspond to efficiently integrable probability distributions. arXiv (2002). https://arxiv.org/abs/quant-ph/0208112

If we have discretized $p(x)$ and $f(x)$ in $2^n$ intervals the first encoding procedure is the following:

1. Intialize $n+1$ qubits:

$$|0\rangle\otimes|0\rangle_n$$

2. Create a $\mathbf{U}_p$ operator for loading $p(x_i)$ array (this will be created using the *load_probability* function from **QQuantLib.DL.data_loading** module). This operator will be applied over the first $n$ qubits in the following way:

$$\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n} = |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n} \tag{1}$$

3. Create a $\mathbf{U}_f$ operator for loading $\sqrt{f(x_i)}$ array (this will be created using the *load_array* function from **QQuantLib.DL.data_loading** module). This operator will act in the following way:

$$\mathbf{U}_f|i\rangle|0\rangle = |i\rangle\left(\sqrt{f(x_i)}|0\rangle + \sqrt{1-f(x_i)}|1\rangle \right) \tag{2}$$

4. Apply the $\mathbf{U}_f$ operator over the $n+1$ qubits:

$$|\Psi\rangle = \mathbf{U}_f\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n}\tag{3}$$

5. Applying equation $(1)$ on $(3)$:
$$|\Psi\rangle = \mathbf{U}_f \left(|0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n}\right) = \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)} \mathbf{U}_f \left(|0\rangle\otimes|i\rangle_{n}\right)$$

5. Applying equation $(2)$:

$$|\Psi\rangle 
= \sum_{i=0}^{2^{n}-1}|i\rangle_{n}\otimes\left(\sqrt{p(x_i)f(x_i)}|0\rangle + \sqrt{p(x_i)(1-f(x_i))}|1\rangle \right) \tag{4}$$

With the above procedure we have encoded the quantity:
$$\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}$$
into the amplitude of the state $|0\rangle$ of the additional qubit. 

In order to see this, we are going to compute the probability of measure the state $|0\rangle$ in the addtional qubit in the equation $(4)$. This will be the sum of the probabilities of measure each posible state $|0\rangle\otimes|i\rangle_n$


$$\mathbf{P}_{|0 \rangle} = \sum_{j=0}^{2^n-1}\mathbf{P}_{|j\rangle_n|0\rangle} \tag{5}$$

Where:

$$\mathbf{P}_{|j\rangle_n|0\rangle} =\left| \; \langle j_n 0|\Psi\rangle \; \right|^2$$

And 

$$\langle j_n 0| = \big( |j\rangle_n \otimes |0\rangle \big) \dagger$$

Using $(4)$ and $\langle 1|0\rangle = 0$:


$$\mathbf{P}_{|j\rangle^n|0\rangle} = \left| \sum_{i=0}^{2^n+1}\sqrt{p(x_i)*f(x_i)}\; _n\langle j|i\rangle_n \langle 0|0\rangle \right|^2 $$

Using  $_n\langle j|i\rangle_n = \delta_{ji}$:

$$\mathbf{P}_{|j\rangle^n|0\rangle} = \left|\sqrt{p(x_j)*f(x_j)} \right|^2 = \left| p(x_j)*f(x_j) \right| \tag{6}$$

So using $(6)$ in $(5)$:

$$\mathbf{P}_{|0 \rangle} = \sum_{j=0}^{2^n-1} \left| p(x_j)*f(x_j) \right| \tag{7}$$

For selecting this encoding procedure in the **Encode** class the the *encoding* attribute of the class must be sett to **0**. For getting the oracle corresponding to the encode the *oracle_encoding_0* method should be used.

**BE AWARE In this case the *encoding_normalization* atribute will be 1.0**


In [ ]:
class_encoding0 = Encoding(array_function=norm_f_x, array_probability=norm_p_x, encoding=0)

print(class_encoding0.encoding)

In [ ]:
class_encoding0.oracle_encoding_0()

In [ ]:
#The desired oracle is stored in the oracle property of the class
c = class_encoding0.oracle
%qatdisplay c --depth 0
#Additionally we can get the U_p operator
c = class_encoding0.p_gate
%qatdisplay c --depth 1
#And we can get the U_f operator
c = class_encoding0.function_gate
%qatdisplay c --depth 1

In order to see that all works properly we can compute the probability of getting the the state $|0\rangle$ into the additionbal qubit. This can be done using the *get_results* from **QQuantLib.utils.data_extracting** module.

In [ ]:
encoding0_results,_,_,_ = get_results(class_encoding0.oracle, linalg_qpu=linalg_qpu, qubits=[class_encoding0.oracle.arity-1])

In [ ]:
encoding0_results

In [ ]:
#The solution should be equal to the scalar product of the loaded probability and function (this is the normalised ones)
print("Encoding normalisation: {}".format(class_encoding0.encoding_normalization))
np.isclose(
    encoding0_results['Probability'].iloc[0]*class_encoding0.encoding_normalization, 
    np.sum(norm_p_x*norm_f_x)
)

In [ ]:
#Additionally undoing the normalisation constants we get the desired Rieman sum!!
np.isclose(
    encoding0_results['Probability'].iloc[0] * class_encoding0.encoding_normalization *  p_x_normalisation * f_x_normalisation, 
    np.sum(p_x*f_x)
)

### uniform distributions for probability

If not *array_probability* is provided to the class an uniform distribution (based on the provided *array_function*) is used. In this case Haddamard gates will be applyied to the first $n$ qubits.  In this case equation $(4)$ can be written as:

$$|\Psi\rangle 
= \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n}\otimes\left(\sqrt{f(x_i)}|0\rangle + \sqrt{1-f(x_i)}|1\rangle \right) \tag{4.2}$$

And equation $(7)$ can be written as:

$$\mathbf{P}_{|0 \rangle} = \frac{1}{2^n} \sum_{j=0}^{2^n-1} \left| f(x_j) \right| \tag{7.2}$$

The uniform probability distribution can be used for computing pure integrals of functions.

**BE AWARE In this case the *encoding_normalization* atribute will be: $2^n$**

In [ ]:
#Raise an error because probability is not properly normalised
class_encoding0 = Encoding(array_function=norm_f_x, encoding=0)
print(class_encoding0.encoding)
class_encoding0.oracle_encoding_0()

In [ ]:
#The desired oracle is stored in the oracle property of the class
c = class_encoding0.oracle
%qatdisplay c --depth 0
#Additionally we can get the U_p operator
c = class_encoding0.p_gate
%qatdisplay c --depth 1
#And we can get the U_f operator
c = class_encoding0.function_gate
%qatdisplay c --depth 1

In [ ]:
encoding0_results,_,_,_ = get_results(class_encoding0.oracle, linalg_qpu=linalg_qpu, qubits=[class_encoding0.oracle.arity-1])

In [ ]:
encoding0_results

In [ ]:
#Now using (7.2) we can compute the sum of the terms of the normalised funtcion array
print("Encoding normalisation: {}".format(class_encoding0.encoding_normalization))
np.isclose(
    encoding0_results['Probability'].iloc[0] * class_encoding0.encoding_normalization,
    np.sum(norm_f_x)
)

In [ ]:
#Additionally undoing the normalisation constant we get the desired Rieman sum!!
np.isclose(
    encoding0_results['Probability'].iloc[0] * class_encoding0.encoding_normalization * f_x_normalisation,
    np.sum(f_x)
)

### BE AWARE!! Encoding problems of Procedure 0

One of the main problems of the first encoding procedure is that the function must be positive definite. If any of the values of the input array $f(x_i)$ is negative their contribution to the final sum $\mathbf{P}_{|0 \rangle} = \sum_{j=0}^{2^n-1} \left| p(x_j)*f(x_j) \right|$ will not be done in the proper way.

Instead of raising an Error in this type of situations we prefer raising a Warning and allowing the complete oracle creation when the *oracle_encoding_0* method is called.

For take into account these cases the operator  $\mathbf{U}_f$ (step 3 of the procedure) wil encode $\sqrt{|f(x_i)|}$ instead of $\sqrt{f(x_i)}$. 

Here we developed an explicit example:

In [ ]:
a = np.pi-np.pi/2.0
b = np.pi+np.pi/4.0
#number of qbits
n = 6
#Our domain
domain_x = np.linspace(a, b, 2**n)
#Our discretized probability distribution
p_x = domain_x
#Our discretized function
f_x = np.sin(domain_x)
plt.plot(domain_x, f_x, 'o')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['p(x)', 'f(x)'])


#Normalisations!!!
p_x_normalisation = np.sum(p_x) + 1e-8
norm_p_x = p_x / p_x_normalisation
print("Probability properly normalised?: {}".format(np.sum(norm_p_x) <= 1.0))
f_x_normalisation = np.max(f_x) +  1e-8
norm_f_x = f_x / f_x_normalisation
print("Function properly normalised?: {}".format(np.max(norm_f_x) <= 1.0))

In [ ]:
#Raise an error because probability is not properly normalised
class_encoding0 = Encoding(array_function=norm_f_x, array_probability=norm_p_x, encoding=0)
class_encoding0.oracle_encoding_0()

In [ ]:
encoding0_results,_,_,_ = get_results(class_encoding0.oracle, linalg_qpu=linalg_qpu, qubits=[class_encoding0.oracle.arity-1])

In [ ]:
encoding0_results

In [ ]:
#Using equation (7) we compute the desired integral and this wil FAIL
print("Encoding normalisation: {}".format(class_encoding0.encoding_normalization))
np.isclose(
    encoding0_results['Probability'].iloc[0] * class_encoding0.encoding_normalization *  p_x_normalisation * f_x_normalisation,
    np.sum(p_x*f_x)
)

In [ ]:
#This is because with the procedure 0 we encoded the sum of the absolute values:

np.isclose(
    encoding0_results['Probability'].iloc[0] * class_encoding0.encoding_normalization *  p_x_normalisation * f_x_normalisation,
    np.sum(np.abs(p_x*f_x)) # BE AWARE NOW WE USE ABSOLUTE VALUES
)

### 2.2 Encoding Procedure 1.

As explained at the end of the before section when the function $f(x)$ is not strictly defined positive the codified value it is not the desired one. 

A second encoding procedure was deloped for solving this issue:

1. Initialize a $n+2$ qbits state.
$$|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}$$

2. Apply the uniform distribution over the $n$ qbits state:

$$\big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = |0\rangle \otimes |0\rangle \otimes H^{\otimes n}|0\rangle_{n}=
\frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|0\rangle \otimes |0\rangle \otimes|i\rangle_{n}\tag{8}$$

3. Create the two operators $\mathbf{U}_p$ and $\mathbf{U}_f$ for encoding $p(x)$ and $f(x)$, respectively. These operators will be created using the *load_array* function from **QQuantLib.DL.data_loading** module. These operators acts in the following way:

$$\mathbf{U}_p|i\rangle_n \otimes |0\rangle = |i\rangle_n \otimes \left(p(x_i)|0\rangle + (1-p(x_i))|1\rangle \right) \tag{9}$$
$$\mathbf{U}_f|i\rangle_n \otimes |0\rangle = |i\rangle_n \otimes \left(f(x_i)|0\rangle + (1-f(x_i))|1\rangle \right) \tag{10}$$

4. Apply the operator $\mathbf{U}_p$  on one of the additional qbits and in the other $n$ registers on the equation $(8)$:


$$ \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = \left( I \otimes \mathbf{U}_p \right) \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|0\rangle \otimes |0\rangle \otimes|i\rangle_{n}= \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|0\rangle \otimes \mathbf{U}_p \left(|0\rangle \otimes|i\rangle_{n}\right) \tag{11}$$

5. Applying $(9)$ in $(11)$:


$$ \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|0\rangle  \otimes \big(p(x_i)|0\rangle + (1-p(x_i))|1\rangle \big) \otimes |i\rangle_n \tag{12}$$

6. Apply the operator $\mathbf{U}_f$ to the other additional qbit and in the other $n$ registers on the equation $(12)$:

$$ \left(\mathbf{U}_f \otimes I  \right) \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} \mathbf{U}_f \big(|0\rangle \otimes |i\rangle_n \big) \otimes \big(p(x_i)|0\rangle + (1-p(x_i))|1\rangle \big) \tag{13}$$

7. Using $(10)$ in $(13)$:

$$ \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} |i\rangle_n \otimes \big(f(x_i)|0\rangle + (1-f(x_i))|1\rangle \big) \otimes \big(p(x_i)|0\rangle + (1-p(x_i))|1\rangle \big) \tag{14}$$

8. Reordering $(14)$

$$ \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |i\rangle_n \otimes |0\rangle \otimes |0\rangle + \big( ... |i\rangle_n \otimes |0\rangle \otimes |1\rangle \big) +  \big( ... |i\rangle_n \otimes |1\rangle \otimes |0\rangle \big) + \big( ... |i\rangle_n \otimes |1\rangle \otimes |1\rangle \big)$$

9. The terms with $|i\rangle_n \otimes |0\rangle \otimes |1\rangle$, $|i\rangle_n \otimes |1\rangle \otimes |0\rangle$ and $|i\rangle_n \otimes |1\rangle \otimes |1\rangle$ are not interested for us so we can omit them into the above expresion so we arrive to $(15)$:

$$ \left(\mathbf{U}_f \otimes I  \right) \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |i\rangle_n \otimes |0\rangle \otimes |0\rangle \; + \; ... \tag{15}$$

10. Finally the uniform distribution is applied again to the $n$ first qubits (so applying uniform distribution to $(15)$):

$$|\Psi\rangle =  \big(I \otimes I \otimes H^{\otimes n}\big) \left(\mathbf{U}_f \otimes I  \right) \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = 
\frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |0\rangle \otimes |0\rangle \otimes H^{\otimes n} |i\rangle_n \; + \; ...  \tag{16}$$

11 . The uniform distribution acting over any $|i\rangle_n$ state can be expresed as:

$$H^{\otimes n}|i\rangle_{n} = \frac{1}{\sqrt{2^n}}\sum_{j=0}^{2^{n}-1} (-1)^{ij}|j\rangle_n=\frac{1}{\sqrt{2^n}}[|0\rangle_n + \sum_{j=1}^{2^{n}-1} (-1)^{ij}|j\rangle_n] \tag{17}$$ 

12. Finally applying $(17)$ to $(16)$ we arrive to the final state of the procedure:


$$|\Psi\rangle =  \big(I \otimes I \otimes H^{\otimes n}\big) \left(\mathbf{U}_f \otimes I  \right) \left( I \otimes \mathbf{U}_p \right)  \big(I \otimes I \otimes H^{\otimes n}\big)\big(|0\rangle \otimes |0\rangle \otimes|0\rangle_{n}\big) = \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |0\rangle \otimes |0\rangle \otimes |0\rangle_n \; + \; ... \tag{18}$$

13. The desired Riemman sum is then codified into the amplitude of the state: $|0\rangle \otimes |0\rangle \otimes |0\rangle_n$

$$\mathbf{P}_{|0\rangle_n|0\rangle|0\rangle} = \left| \; \langle 0 0 0_n |\Psi\rangle \; \right|^2 \tag{19}$$

Where: 
$$\langle 0 0 0_n | = \big( |0\rangle \otimes |0\rangle \otimes |0\rangle_n\big)\dagger $$

14. So using $(18$) into $(19)$:

$$\mathbf{P}_{|0\rangle_n|0\rangle|0\rangle} = \left| \; \langle 0 0 0_n |\frac{1}{2^n} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |0\rangle \otimes |0\rangle \otimes |0\rangle_n \right|^2 = \left| \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) \right|^2 \tag{20}$$

15. With this encoding procedure the desired integral will be then:

$$\sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) = {2^n} \sqrt{\mathbf{P}_{|0\rangle_n|0\rangle|0\rangle}} \tag{21}$$

For selecting this encoding procedure in the **Encode** class the the *encoding* attribute of the class must be sett to **1**. For getting the oracle corresponding to the encode the *oracle_encoding_1* method should be used.

**BE AWARE In this case the *encoding_normalization* atribute will be: $2^n$**

**BE AWARE** In this encoding the *array_probability* **CAN NOT BE NONE**. An error will be raised when *oracle_encoding_1* methods is used and *array_probability* is not provided!!

In [ ]:
#Encoding type 1
class_encoding1 = Encoding(array_function=norm_f_x, array_probability=norm_p_x, encoding=1)

In [ ]:
#execute the correspondient encoding method
class_encoding1.oracle_encoding_1()

In [ ]:
#The desired oracle is stored in the oracle property of the class
c = class_encoding1.oracle
%qatdisplay c --depth 0
#Additionally we can get the U_p operator
c = class_encoding1.p_gate
%qatdisplay c --depth 0
#And we can get the U_f operator
c = class_encoding1.function_gate
%qatdisplay c --depth 0

In order to see that all works properly we can compute the probability of getting the the state $|0\rangle \otimes |0\rangle \otimes |0\rangle_n$. This can be done using the *get_results* from **QQuantLib.utils.data_extracting** module.

In [ ]:
encoding1_results,_,_,_ = get_results(class_encoding1.oracle, linalg_qpu=linalg_qpu)

In [ ]:
encoding1_results

In [ ]:
#The solution should be equal to the scalar product of the loaded probability and function (this is the normalised ones)
print("Encoding normalisation: {}".format(class_encoding1.encoding_normalization))
measure_0 = encoding1_results['Probability'].iloc[0]
np.isclose(
    np.sqrt(measure_0) * class_encoding1.encoding_normalization, 
    np.sum(norm_p_x*norm_f_x)
)

In [ ]:
#Additionally undoing the normalisation constants we get the desired Rieman sum!!
np.isclose(
    np.sqrt(measure_0) * class_encoding1.encoding_normalization *  p_x_normalisation * f_x_normalisation, 
    np.sum(p_x*f_x)
)

In [ ]:
#If array_probability is not provide and encoding 1 is used an error is raised

#Encoding type 1
class_encoding1 = Encoding(array_function=norm_f_x, encoding=1)
#Error will be raised
class_encoding1.oracle_encoding_1()

### 2.3 Encoding Procedure 2.

Main problem of the encoding 1 procedure is that 2 additional qubits are needed. A third procedure with only $n+1$ qubits and that properly computes the desired sum is provided now:

1. Intialize $n+1$ qubits:

$$|0\rangle\otimes|0\rangle_n$$

2. Create a $\mathbf{U}_p$ operator for loading $p(x_i)$ array (this will be created using the *load_probability* function from **QQuantLib.DL.data_loading** module). This operator will be applied over the first $n$ qubits in the following way:

$$\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n} = |0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n} \tag{22}$$

3. Create a $\mathbf{U}_f$ operator for loading $f(x_i)$ array (this will be created using the *load_array* function from **QQuantLib.DL.data_loading** module). This operator will act in the following way:

$$\mathbf{U}_f|i\rangle_n \otimes |0\rangle = |i\rangle_n \otimes \big(f(x_i)|0\rangle +  \left( 1-f(x_i) \right)|1\rangle \big) \tag{23}$$

4. Apply the $\mathbf{U}_f$ operator over $n+1$ qubits:

$$\mathbf{U}_f\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n}\tag{24}$$

5. Applying equation $(22)$ on $(24)$:
$$\mathbf{U}_f\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n} = \mathbf{U}_f \left(|0\rangle\otimes \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)}|i\rangle_{n}\right) = \sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)} \mathbf{U}_f \big(|0\rangle\otimes|i\rangle_{n}\big) \tag{25}$$

6. Applying equation $(23)$ on $(25)$:

$$\mathbf{U}_f\left(I\otimes \mathbf{U}_p \right)|0\rangle\otimes|0\rangle_{n} = \sum_{i=0}^{2^{n}-1}|i\rangle_{n}\otimes\left(\sqrt{p(x_i)}f(x_i)|0\rangle + \sqrt{p(x_i)}(1-f(x_i))|1\rangle \right) \tag{26}$$

7. Finally the transpose of the $\mathbf{U}_p$ operator will be applied over the first $n$ qbits:

$$|\Psi \rangle = \left(I\otimes \mathbf{U}_p \dagger \right) \mathbf{U}_f \left(I\otimes \mathbf{U}_p \right) |0\rangle\otimes|0\rangle_{n} \tag{27}$$

8. Using $(26)$ into $(27)$:

$$|\Psi \rangle = \left(I\otimes \mathbf{U}_p \dagger \right) \sum_{i=0}^{2^{n}-1}|i\rangle_{n}\otimes\left(\sqrt{p(x_i)}f(x_i)|0\rangle + \sqrt{p(x_i)}(1-f(x_i))|1\rangle \right)$$
$$|\Psi \rangle = \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)}f(x_i) \left( I\otimes \mathbf{U}_p \dagger \right) \big( |0\rangle \otimes |i\rangle_{n} \big) + \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)}(1-f(x_i)) \left( I\otimes \mathbf{U}_p \dagger \right) \big( |1\rangle \otimes |i\rangle_{n} \big)$$

9. We are interested only in $|0\rangle \otimes |i\rangle_{n}$ so we don't need to take into acount other terms, so above expresion can be expresed as:

$$|\Psi \rangle = \left(I\otimes \mathbf{U}_p \dagger \right) \mathbf{U}_f \left(I\otimes \mathbf{U}_p \right) |0\rangle\otimes|0\rangle_{n} = \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)}f(x_i) \left( I\otimes \mathbf{U}_p \dagger \right) \big( |0\rangle \otimes |i\rangle_{n} \big) \; + \; ... = \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)}f(x_i) |0\rangle \otimes \mathbf{U}_p \dagger |i\rangle_{n} \; + \; ... \tag{28}$$

10. It can be proved that: 

$$\mathbf{U}_p \dagger |i\rangle_{n} = \sqrt{p(x_i)} |0\rangle_{n} + \sum_{j=1}^{2^n-1} u_{ij}|j\rangle_{n} \tag{29}$$

where $u_{ij}$ are coeficients from the $\mathbf{U}_p$ operator when $j\gt 1$ (we are not interested in them!!)

11. So plugin $(29)$ into $(28)$:


$$|\Psi \rangle = \sum_{i=0}^{2^{n}-1} \sqrt{p(x_i)}f(x_i) \sqrt{p(x_i)} |0\rangle \otimes |0\rangle_{n} + \sum_{j=1}^{2^n-1} \sqrt{p(x_i)}f(x_i) u_{ij}|0\rangle \otimes |j\rangle_{n} \; + \; ...$$

12. We are only interested in the $|0\rangle \otimes |0\rangle_{n}$ so the other terms will be not taking into account. So our final procedure will be:


$$|\Psi \rangle = \left(I\otimes \mathbf{U}_p \dagger \right) \mathbf{U}_f \left(I\otimes \mathbf{U}_p \right) |0\rangle\otimes|0\rangle_{n} = \sum_{i=0}^{2^{n}-1} p(x_i) f(x_i) |0\rangle \otimes |0\rangle_{n} \; + \; ... \tag{30}$$

13. The desired Riemman sum is then codified into the amplitude of the state: $|0\rangle \otimes |0\rangle_n$

$$\mathbf{P}_{|0\rangle_n|0\rangle} = \left| \; \langle 0 0_n |\Psi\rangle \; \right|^2 \tag{31}$$

Where: 
$$\langle 0 0_n | = \big( |0\rangle \otimes |0\rangle_n\big)\dagger $$

14. So using $(30$) into $(31)$:

$$\mathbf{P}_{|0\rangle_n|0\rangle} = \left| \; \langle 0 0_n |\sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) |0\rangle \otimes |0\rangle_n \right|^2 =  \left|  \sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) \right|^2 \tag{32}$$

15. With this encoding procedure the desired integral will be then:

$$\sum_{i=0}^{2^{n}-1} p(x_i)f(x_i) = \sqrt{\mathbf{P}_{|0\rangle_n|0\rangle}} \tag{33}$$

For selecting this encoding procedure in the **Encode** class the the *encoding* attribute of the class must be set to **2**. For getting the oracle corresponding to the encode the *oracle_encoding_2* method should be used.

**BE AWARE In this case the *encoding_normalization* atribute will be: 1.0**

In [ ]:
#Encoding type 2
class_encoding2 = Encoding(array_function=norm_f_x, array_probability=norm_p_x, encoding=2)

In [ ]:
#execute the correspondient encoding method
class_encoding2.oracle_encoding_2()

In [ ]:
#The desired oracle is stored in the oracle property of the class
c = class_encoding2.oracle
%qatdisplay c --depth 0
#Additionally we can get the U_p operator
c = class_encoding2.p_gate
%qatdisplay c --depth 0
#And we can get the U_f operator
c = class_encoding2.function_gate
%qatdisplay c --depth 0

In [ ]:
encoding2_results,_,_,_ = get_results(class_encoding2.oracle, linalg_qpu=linalg_qpu)

In [ ]:
#The solution should be equal to the scalar product of the loaded probability and function (this is the normalised ones)
measure_0_enc2 = encoding2_results['Probability'].iloc[0]
print("Encoding normalisation: {}".format(class_encoding2.encoding_normalization))
np.isclose(
    np.sqrt(measure_0_enc2) * class_encoding2.encoding_normalization,
    np.sum(norm_p_x*norm_f_x)
)

In [ ]:
#Additionally undoing the normalisation constants we get the desired Rieman sum!!
np.isclose(
    np.sqrt(measure_0_enc2) * class_encoding2.encoding_normalization *  p_x_normalisation * f_x_normalisation, 
    np.sum(p_x*f_x)
)

### uniform distributions for probability

If not *array_probability* is provided to the class an uniform distribution (based on the provided *array_function*) is used. In this case Haddamard gates will be applyied to the first $n$ qubits. In this case equation $(30)$ we change $\mathbf{U}_p$ by $H^{\otimes n}$:

$$|\Psi \rangle = \left(I\otimes H^{\otimes n} \right) \mathbf{U}_f \left(I\otimes H^{\otimes n} \right) |0\rangle\otimes|0\rangle_{n} = \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f(x_i) |0\rangle \otimes |0\rangle_{n} \; + \; ... \tag{30.2}$$

And equation $(32)$ can be written as:

$$\mathbf{P}_{|0\rangle_n|0\rangle} = \left|  \frac{1}{2^n} \sum_{i=0}^{2^{n}-1} f(x_i) \right|^2 \tag{32.2}$$

With the uniform probability distribution we can compute the Riemman sum of a function $f(x)$ (so we can compute their integral!!):

$$\sum_{i=0}^{2^{n}-1} f(x_i) = 2^n \sqrt{\mathbf{P}_{|0\rangle_n|0\rangle}}$$

**BE AWARE In this case the *encoding_normalization* atribute will be: $2^n$**

In [ ]:
#Encoding type 2. Uniform distribution
class_encoding2 = Encoding(array_function=norm_f_x, encoding=2)
#execute the correspondient encoding method
class_encoding2.oracle_encoding_2()
#The desired oracle is stored in the oracle property of the class
c = class_encoding2.oracle
%qatdisplay c --depth 0
#Additionally we can get the U_p operator
c = class_encoding2.p_gate
%qatdisplay c --depth 1
#And we can get the U_f operator
c = class_encoding2.function_gate
%qatdisplay c --depth 0

In [ ]:
encoding2_results,_,_,_ = get_results(class_encoding2.oracle, linalg_qpu=linalg_qpu)

In [ ]:
#Now using (30.2) we can compute the sum of the terms of the normalised funtcion array
measure_0_enc2 = encoding2_results['Probability'].iloc[0]
print("Encoding normalisation: {}".format(class_encoding2.encoding_normalization))
np.isclose(
    np.sqrt(measure_0_enc2) * class_encoding2.encoding_normalization ,
    np.sum(norm_f_x) #Only the normalised function is needed
)

In [ ]:
#Additionally undoing the normalisation constant we get the desired Rieman sum!!
np.isclose(
    np.sqrt(measure_0_enc2) * class_encoding2.encoding_normalization * f_x_normalisation, 
    np.sum(f_x)
)